# Silver層テーブル作成（Bronze to Silver）

このノートブックでは、Bronze層のテーブルを結合・クリーニングしてSilver層の統合テーブルを作成します。

## 目的
- **データ品質向上**: NULL値の除外とデータ検証
- **データ統合**: 複数のマスタテーブルとの結合
- **スキーマ標準化**: データ型の統一と最適化

## 処理内容
1. Bronze層の各テーブル読み込み
2. データ品質チェックとフィルタリング
3. 必要なテーブル間でのJOIN処理
4. エンリッチされたSilver層テーブルの作成

## Silver層の特徴
- クリーンでビジネスルールが適用されたデータ
- 分析可能な形でのデータ統合
- Gold層の基盤となる高品質データ


## 環境設定

カタログとスキーマを設定して、Bronze層のテーブルにアクセスできるようにします。


In [0]:
# 環境変数の定義（前ステップと統一）
catalog_name = "users"              # Unity Catalogのカタログ名
schema = "yukiteru_koide"           # スキーマ名

# カタログとスキーマの切り替え
spark.sql(f"USE CATALOG {catalog_name}")  # カタログを指定
spark.sql(f"USE SCHEMA {schema}")          # スキーマを指定

print(f"環境設定完了: {catalog_name}.{schema}")

## データ統合とエンリッチメント処理

Bronze層のテーブルを読み込み、JOIN処理とデータクリーニングを実行してSilver層テーブルを作成します。

### 処理ステップ
1. Bronze層テーブルの読み込み
2. データ品質チェック（数量 > 0）
3. 日付形式の標準化
4. LEFT JOINによるマスタ情報の付与


In [0]:
# Step 1: Bronze層テーブルの読み込み
print("📋 Bronze層テーブルを読み込み中...")
df_transactions = spark.table(f"{catalog_name}.{schema}.transactions")   # 取引履歴
df_customers = spark.table(f"{catalog_name}.{schema}.customers")         # 顧客マスタ
df_products = spark.table(f"{catalog_name}.{schema}.products")           # 商品マスタ
df_stores = spark.table(f"{catalog_name}.{schema}.stores")               # 店舗マスタ

print(f"✅ 読み込み完了:")
print(f"  - 取引履歴: {df_transactions.count()} レコード")
print(f"  - 顧客: {df_customers.count()} レコード")
print(f"  - 商品: {df_products.count()} レコード")
print(f"  - 店舗: {df_stores.count()} レコード")

# Step 2: データ統合とクリーニング
print("\n🔄 データ統合処理を実行中...")
from pyspark.sql.functions import to_date, col

df_silver = (
    df_transactions
    .withColumn("transaction_date", to_date(col("transaction_date")))  # 日付型に変換
    .filter(col("quantity") > 0)                                       # 数量が0より大きいもののみ
    .join(df_customers, "customer_id", "left")                         # 顧客情報を結合
    .join(df_products, "product_id", "left")                           # 商品情報を結合
    .join(df_stores, "store_id", "left")                               # 店舗情報を結合
)

print(f"🔍 フィルタリング後のレコード数: {df_silver.count()}")

# Step 3: Silver層テーブルとして保存
print("\n💾 Silver層テーブル保存中...")
df_silver.write.format("delta").mode("overwrite").saveAsTable("transactions_enriched")

print("✅ transactions_enriched テーブル作成完了！")
print(f"📊 最終レコード数: {df_silver.count()}")
print(f"📋 カラム数: {len(df_silver.columns)}")

# データサンプルの表示
print("\n📋 作成されたテーブルのサンプル:")
df_silver.select("transaction_id", "customer_id", "product_name", "store_name", "total_price").show(5)
